In [ ]:
import numpy
import string
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

Using TensorFlow backend.


In [ ]:
# load ascii text and covert to lowercase
filename = "Index_fund_all.xlsx"
df = pd.read_excel(filename)
df.head() 

,Unnamed: 0,accession#,filing_year,principal_strategies
0,0,0000067590-10-000105,2010,. the fund intends under normal circumstances ...
1,1,0000067590-10-000829,2010,. the fund intends under normal circumstances ...
2,2,0000088053-10-000550,2010,references to the fund may refer to actions ta...
3,3,0000088053-10-000554,2010,references to the fund may refer to actions ta...
4,4,0000356476-10-000108,2010,the portfolio will invest at least 80% of its ...


In [ ]:
# create mapping of unique chars to integers
flat_list = []
raw_text = df.principal_strategies.tolist()
for item in raw_text:
    words = str(item).split()
    for i in words:
        i = i.lower().translate(str.maketrans('', '', string.punctuation))
        flat_list.append(i.strip())
        
raw_text = ' '.join(flat_list)

chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))

In [ ]:
# summarize the loaded data
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)

# prepare the dataset of input to output pairs encoded as integers
seq_length = 120
dataX = []
dataY = []

for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])

n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)

Total Characters:  18217952
Total Vocab:  70
Total Patterns:  18217832


In [ ]:
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))

# normalize
X = X / float(n_vocab)

# one hot encode the output variable
y = np_utils.to_categorical(dataY)

# define the LSTM model
model = Sequential()
model.add(LSTM(256, 
               input_shape=(X.shape[1], X.shape[2]), 
               return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentropy', 
              optimizer='adam')

# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

# fit the model
model.fit(X, y, epochs=50, batch_size=64, callbacks=callbacks_list)

In [ ]:
# load the network weights
filename = "weights-improvement-05-0.6441-bigger.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[1000]

print "Seed:"
print (''.join([int_to_char[value] for value in pattern]))

normal circumstances, ...
18     the fund seeks its investment objective by inv...
19               


In [ ]:
# generate characters
for i in range(100):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]

In [ ]:
# sys.stdout.flush()